# Athuga niðurstöður

In [1]:
import os
from pathlib import Path
from pprint import pprint

import frontend.core as c
import frontend.bulk as b

%matplotlib notebook
c.THREADS = 10
c.MEMORY = 17179869184 # 16GB

working_dir = Path('/work/haukurpj')

IS = c.Lang.IS
EN = c.Lang.EN

langs = [IS, EN]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/staff/haukurpj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2019-12-20 09:58:18,019 - frontend - INFO - Initialized


In [2]:
models = ['en-is-improved', 'is-en-improved', 'is-en-mideind-v2', 'en-is-mideind-v2', 'is-en-mono']
models.sort()
model_dirs = [working_dir.joinpath(model).joinpath('binarised') for model in models]

In [3]:
def blue_score(model_dir):
    !cat {model_dir.joinpath('translated.bleu')}

In [4]:
print([blue_score(model_dir) for model_dir in model_dirs])

cat: /work/haukurpj/en-is-improved/binarised/translated.bleu: No such file or directory
BLEU = 48.56, 69.7/51.5/42.7/36.3 (BP=1.000, ratio=1.003, hyp_len=37548, ref_len=37435)
cat: /work/haukurpj/is-en-improved/binarised/translated.bleu: No such file or directory
BLEU = 53.82, 75.1/57.5/47.9/41.1 (BP=0.997, ratio=0.997, hyp_len=40766, ref_len=40890)
BLEU = 52.99, 75.0/57.2/47.4/40.7 (BP=0.988, ratio=0.988, hyp_len=40395, ref_len=40890)
[None, None, None, None, None]


In [21]:
def peeks(paths, length: int = 10):
    generators = [b.peek(paths[model_name], length) for model_name in paths]
    remaining = True
    while remaining:
        for index, model_name in enumerate(paths):
            try:
                sentence = next(generators[index])
                yield f'{model_name:<20}: {sentence}'
            except StopIteration:
                return

In [15]:
def get_model_translated(model_name):
    model_dir = working_dir.joinpath(model_name).joinpath('binarised')
    translated = list(model_dir.glob("translated.is"))
    if translated:
        return translated[0]
    else:
        return list(model_dir.glob("translated.en"))[0]

In [16]:
check = {
    'source': working_dir.joinpath('process').joinpath('parice-test-final.is'),
    'target': working_dir.joinpath('process').joinpath('parice-test-final.en'),
    'is-en-mideind-v2': get_model_translated('is-en-mideind-v2'),
    'is-en-mono': get_model_translated('is-en-mono')
}

In [22]:
print(*peeks(check, 30))

source              : • 6 km fyrir bifhjól í flokki 2 ( slagrými hreyfils ≥ 150 cc , vmax _lt_ 130 km/klst. ) ,
 target              : • 6 km for category 2 motorcycle ( engine capacity ≥ 150 cc , vmax _lt_ 130 km / h ) ,
 is-en-mideind-v2    : • 6 km for category 2 motorcycle ( engine capacity ≥ 150 cc , vmax _lt_ 130 km / h ) , 
 is-en-mono          : • 6 km for category 2 motorcycle ( engine capacity ≥ 150 cc , vmax _lt_ 130 km / h ) , 
 source              : e. aðgerð varðandi sameiginlega fræðslu e-liður 1. mgr. 5. gr .
 target              : e. common learning article 5 ( 1 ) ( e )
 is-en-mideind-v2    : e. common learning article 5 ( 1 ) ( e ) . 
 is-en-mono          : e. common learning article 5 ( 1 ) ( e ) 
 source              : mælingar á reykþéttni útblásturslofts við hröðun ( frá hægagangi og upp í marksnúningshraða , án álags ) .
 target              : measurement of exhaust gas opacity with free acceleration ( no load from idling up to cut-off speed ) .
 is-en-mideind-v